In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import mysmith as mys
import myplot as myp
from skrf import Frequency
from skrf.network import *
%matplotlib inline

# Datos 
rg = 100
Rl = 200
rg = 100
fo = 500e6
wo = 2*np.pi*fo


Qm = (Rl/rg-1)**0.5      # Qm para conversion de Rl a rg
Bp = Qm/Rl                # Caclulo de la sustencia paralelo
Bpp = Bp*(1+(1/Qm**2))   # Conversion sustencia paralelo
Xs = 1/ Bpp              # Reluctancia de sintonia

# Ahora calculando en la hoja de calculo 
f =  np.arange(200e6,800e6,1e6)                   # Defino el array de frecuencia
w =  2*np.pi*f

# Alternativa 2
C2 = Bp/wo
L2 = Xs/wo
# LTspice
print('.param L2 =  {:1.4e} '.format(L2) )
print('.param C2 =  {:1.4e} '.format(C2) )
print('.param Rl =  {:1.4e} '.format(Rl) )
print('.param rg =  {:1.4e} '.format(rg) )

# Grafico de Zin
Zi = 1/(1j*2*np.pi*f*C1) + 1/(1/Rl + 1/(1j*2*np.pi*f*L1) )  

myp.PlotCpx(f,Zi , labelx = 'Frecuencia/Hz', labely = '$Z_{in}$')
plt.savefig('figures/ejemplo2a.png')
plt.close()

################################################
freq = Frequency(fo/1e6,fo/1e6,1,'mhz')
z0   = 100                                  # impedancia de referencia

# Defino el valor de capacidad paralelo
C2p = np.linspace(C2/100, C2, 1000)

# Defino el valor de inductancia serie
L2s = np.linspace(L2/100, L2, 1000)

nw1 =  mys.ZL(freq,Rl,z0)
nw   = nw1 

for c in C2p:
    nwa = mys.Zparal(freq,1j*mys.Xc(freq.f[0],c),z0)
    nw2  = cascade(nwa,nw1)
    nw   = stitch(nw,nw2 )    

for l in L2s:
    nwa = mys.Yserie(freq,1j*mys.Bl(freq.f[0],l),z0)
    nw3 = cascade(nwa,nw2)
    nw   = stitch(nw,nw3 )

mys.printSmith(nw, c = 'r', m = ' ')    
mys.printSmith(nw1, c = 'g', m = 'o')    
mys.printSmith(nw2, c = 'b', m = 'o')    
mys.printSmith(nw3, c = 'y', m = 'o')  

plt.savefig('figures/ejemplo2bsmitha.png')
plt.show()
